In [1]:
from ont_fast5_api.fast5_interface import get_fast5_file
import pandas as pd
from Settings import *
import LSTMAutoencoder
import numpy as np
from numpy.linalg import norm
import time

1 - 00014eb4-4e2c-4087-ac24-57dea735b7b4

2 - 601f7bc7-3c09-4a78-a9b7-097bddcde809

3 - 1b6939ba-4a35-4696-bc6f-2ddb3368266e

4 - 95c03c50-57a5-4092-9a5f-87182d06a12c

5 - b6d8845b-3eec-42f6-9510-b8ba1fc1ec44


1 <--> 2    90.1

3 <--> 4    91.2

4 <--> 5    95.9 or 90.1


In [2]:
filepath = "../../similar_testdata/similar_squiggles.fast5"

def loadReads():

    print("reading squiggles file...")
    reads = []
    ids = []
    with get_fast5_file(filepath, mode="r") as f5:
        for read in f5.get_reads():
            if read.read_id in subset:
                raw_data = read.get_raw_data()
                reads.append(raw_data)
                ids.append(read.read_id)

    return reads, ids

In [3]:
squiggles, ids = loadReads()
ids

reading squiggles file...


['00014eb4-4e2c-4087-ac24-57dea735b7b4',
 '1b6939ba-4a35-4696-bc6f-2ddb3368266e',
 '601f7bc7-3c09-4a78-a9b7-097bddcde809',
 '95c03c50-57a5-4092-9a5f-87182d06a12c',
 'b6d8845b-3eec-42f6-9510-b8ba1fc1ec44']

In [4]:
model = LSTMAutoencoder.LSTM()
model.load_state_dict(torch.load(model_path))
model.eval()            #setting model to "predict mode"

FileNotFoundError: [Errno 2] No such file or directory: 'model_weights.pt'

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
stateHs = []
stateCs = []
for i in range(len(ids)):
    print(f"embedding {i}...")
    s = time.time()

    x = torch.tensor(scaler.fit_transform(squiggles[i].reshape(-1, 1))).squeeze(-1)
    x = x.type(torch.FloatTensor)


    state_h = torch.zeros(1, hidden_size)
    state_c = torch.zeros(1, hidden_size)

    with torch.no_grad():
        for value in x:
            pred, (state_h, state_c) = model(value, (state_h, state_c))

        
    
    stateHs.append(state_h.squeeze(0))
    stateCs.append(state_c.squeeze(0))
    print(f"Took {time.time()-s}s.")

    

embedding 0...
Took 15.374706983566284s.
embedding 1...
Took 21.962628602981567s.
embedding 2...
Took 33.27038836479187s.
embedding 3...
Took 25.172276258468628s.
embedding 4...
Took 14.064485788345337s.


In [ ]:
print(stateCs[1].shape)
print(stateCs[1].squeeze(0))


torch.Size([200])
tensor([ 4.3052e-01, -3.2039e-01, -1.6582e-01, -1.1707e-01,  2.2227e+00,
        -1.2730e-01,  2.3749e-02,  3.2955e+00, -2.2894e-02, -1.3308e-01,
        -2.9422e-02, -9.1946e-02, -1.2397e-02, -3.9789e-02, -7.5253e-02,
         8.0515e-02,  4.8371e-02,  1.8322e-01,  8.9089e-02,  1.2826e-02,
        -1.2472e+00, -8.8285e-02, -8.8368e-02,  2.5516e-01,  1.6373e-01,
         1.6692e-01, -1.4093e-01, -5.5305e-02,  5.3905e-01,  2.1881e-02,
         2.8987e-01,  6.0362e-01,  1.1437e-01,  1.1387e-01, -1.1960e-02,
         6.0928e-02, -3.9376e-01,  8.7013e-02,  9.3056e-02, -7.5721e-03,
        -1.4006e-01,  7.6234e-02,  9.7582e-02, -4.3426e-02,  3.9312e-01,
        -5.3653e-02, -1.3969e-01,  9.7933e-02,  9.6053e-02, -1.2181e+00,
        -3.4187e-04, -6.9184e-02, -7.5671e-03, -8.3132e-02,  3.2274e-01,
         7.0907e-02,  3.4215e-02,  4.7031e-01, -1.5769e-01, -1.2761e-01,
         1.7540e-01, -5.2492e-02, -6.8880e-01, -4.2344e-02, -1.0023e-01,
         1.4647e+00, -2.9742e-02,

In [ ]:
similartiesH = {}
similartiesC = {}

for i in range(len(ids)):
    innerDictH = {}
    innerDictC = {}
    for s in range(len(ids)):
        if s != i:
            innerDictH[ids[s]] = str(norm(stateHs[i]-stateHs[s]))
            innerDictC[ids[s]] = str(norm(stateCs[i]-stateCs[s]))


    similartiesH[ids[i]] = innerDictH.copy()
    similartiesC[ids[i]] = innerDictC.copy()


In [ ]:
import json

with open("hSimilarties.json", "w") as file:
    json.dump(similartiesH, file)

with open("cSimilarties.json", "w") as file:
    json.dump(similartiesC, file)